# RTM Simulation: Diffusive 1-D (Random Walk)

This notebook demonstrates the **diffusive transport regime** in the RTM (Relatividad Temporal Multiescala) framework.

## Theoretical Background

In diffusive transport, a particle undergoes random walk (Brownian motion). The mean first-passage time (MFPT) to traverse a distance $L$ scales as:

$$T \propto L^\alpha \quad \text{with} \quad \alpha = 2$$

This is derived from the diffusion equation. For a 1-D random walk:
- Mean square displacement: $\langle x^2 \rangle = 2Dt$
- To travel distance $L$: $T \propto L^2/D$
- Therefore: $\alpha = 2$

**Expected result:** $\alpha \approx 2.00$

This serves as the **diffusive benchmark** for RTM temporal-relativity tests.

## 1. Setup and Imports

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd

# For reproducibility
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

print("Libraries loaded successfully!")

## 2. Define Simulation Parameters

In [ ]:
# System sizes (target distances)
SYSTEM_SIZES = np.array([5, 10, 20, 40, 80, 160, 320])

# Number of trials per system size
N_TRIALS = 200

# Maximum steps per trial
MAX_STEPS = 2_000_000

print(f"System sizes: {SYSTEM_SIZES}")
print(f"Number of trials per size: {N_TRIALS}")
print(f"Total simulations: {len(SYSTEM_SIZES) * N_TRIALS}")
print(f"\nTheoretical MFPT for each L:")
for L in SYSTEM_SIZES:
    print(f"  L = {L:4d}  →  T_theory = {L**2:,}")

## 3. Define the Random Walk Model

### 1-D Symmetric Random Walk

The walker starts at the origin and takes steps of $\pm 1$ with equal probability:

$$X_{n+1} = X_n + \xi_n, \quad \text{where } P(\xi = +1) = P(\xi = -1) = 0.5$$

The Mean First-Passage Time (MFPT) to reach $\pm L$ from the origin is **exactly** $T = L^2$.

In [ ]:
def random_walk_mfpt(target_distance, max_steps, rng):
    """
    Optimized 1-D random walk using cumulative sum.
    Returns first-passage time to reach ±target_distance.
    """
    chunk_size = min(100_000, max_steps)
    position = 0
    total_steps = 0
    
    while total_steps < max_steps:
        remaining = max_steps - total_steps
        current_chunk = min(chunk_size, remaining)
        
        # Generate steps and compute cumulative positions
        steps = rng.choice(np.array([-1, 1], dtype=np.int32), size=current_chunk)
        cumulative = np.cumsum(steps) + position
        
        # Find first crossing
        crossings = np.where(np.abs(cumulative) >= target_distance)[0]
        
        if len(crossings) > 0:
            return total_steps + crossings[0] + 1
        
        position = cumulative[-1]
        total_steps += current_chunk
    
    return -1

print("Random walk function defined.")

## 4. Visualize a Sample Random Walk

In [ ]:
# Generate a sample walk to visualize
sample_rng = np.random.default_rng(123)
n_steps = 5000
steps = sample_rng.choice([-1, 1], size=n_steps)
trajectory = np.cumsum(steps)

plt.figure(figsize=(12, 4))
plt.plot(trajectory, linewidth=0.5, alpha=0.8)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
plt.xlabel('Step number', fontsize=12)
plt.ylabel('Position', fontsize=12)
plt.title('Sample 1-D Random Walk Trajectory', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Final position after {n_steps} steps: {trajectory[-1]}")
print(f"Max excursion: {np.max(np.abs(trajectory))}")

## 5. Run the Simulation

In [ ]:
# Reset RNG for reproducibility
rng = np.random.default_rng(RANDOM_SEED)

# Storage for results
results = []

print("Running simulations...\n")
print(f"{'L':>6} | {'T_mean':>12} | {'T_theory':>12} | {'Ratio':>8}")
print("-" * 48)

for L in SYSTEM_SIZES:
    times = []
    for _ in range(N_TRIALS):
        fpt = random_walk_mfpt(int(L), MAX_STEPS, rng)
        if fpt > 0:
            times.append(fpt)
    
    times = np.array(times)
    T_mean = np.mean(times)
    T_std = np.std(times, ddof=1)
    T_sem = T_std / np.sqrt(len(times))
    T_theory = L * L
    ratio = T_mean / T_theory
    
    results.append({
        'L': L,
        'T_mean': T_mean,
        'T_std': T_std,
        'T_sem': T_sem,
        'T_theory': T_theory,
        'ratio': ratio
    })
    
    print(f"{L:>6} | {T_mean:>12.2f} | {T_theory:>12.0f} | {ratio:>8.4f}")

df = pd.DataFrame(results)
print("\nSimulation complete!")

## 6. Fit Power Law: $T = T_0 \cdot L^\alpha$

In [ ]:
# Log-transform
log_L = np.log10(df['L'].values.astype(float))
log_T = np.log10(df['T_mean'].values)

# Linear regression
slope, intercept, r_value, p_value, std_err = stats.linregress(log_L, log_T)

alpha = slope
T0 = 10**intercept
R_squared = r_value**2

print("=" * 50)
print("POWER LAW FIT RESULTS")
print("=" * 50)
print(f"\nFitted exponent: α = {alpha:.4f} ± {std_err:.4f}")
print(f"Prefactor: T₀ = {T0:.4f}")
print(f"R² = {R_squared:.6f}")
print(f"p-value = {p_value:.2e}")
print(f"\nExpected (theoretical): α = 2.00")
print(f"Deviation: {abs(alpha - 2.0) / 2.0 * 100:.2f}%")

## 7. Visualization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Log-log scaling
ax1 = axes[0]
ax1.errorbar(df['L'], df['T_mean'], yerr=df['T_sem'], 
             fmt='o', markersize=10, capsize=4, color='blue', 
             label='Simulation data')

L_fit = np.logspace(np.log10(df['L'].min()), np.log10(df['L'].max()), 100)
T_fit = T0 * L_fit**alpha
T_theory = L_fit**2

ax1.plot(L_fit, T_fit, 'r-', linewidth=2, 
         label=f'Fit: α = {alpha:.3f}')
ax1.plot(L_fit, T_theory, 'g--', linewidth=2, alpha=0.7,
         label='Theory: α = 2.00')

ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel('System Size L', fontsize=12)
ax1.set_ylabel('Mean First-Passage Time T', fontsize=12)
ax1.set_title('Diffusive 1-D: T ∝ L² Scaling', fontsize=14)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3, which='both')

# Plot 2: Comparison with theory
ax2 = axes[1]
ax2.scatter(df['T_theory'], df['T_mean'], s=100, c='blue', alpha=0.7)
max_val = max(df['T_theory'].max(), df['T_mean'].max()) * 1.1
ax2.plot([0, max_val], [0, max_val], 'r--', linewidth=2, label='Perfect agreement')
ax2.set_xlabel('Theoretical MFPT (L²)', fontsize=12)
ax2.set_ylabel('Simulated MFPT', fontsize=12)
ax2.set_title('Simulation vs Theory', fontsize=14)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, max_val)
ax2.set_ylim(0, max_val)

plt.tight_layout()
plt.show()

## 8. Why α = 2 for Diffusion?

### Mathematical Derivation

For a 1-D symmetric random walk starting at origin with absorbing boundaries at $\pm L$:

1. **Gambler's ruin problem**: The MFPT satisfies $T(x) = 1 + \frac{1}{2}T(x-1) + \frac{1}{2}T(x+1)$

2. **Boundary conditions**: $T(-L) = T(L) = 0$

3. **Solution**: $T(x) = L^2 - x^2$

4. **Starting from origin**: $T(0) = L^2$

This gives **exactly** $\alpha = 2$, which our simulation confirms!

### Physical Interpretation

In diffusion, the walker doesn't "know" which direction to go. The quadratic scaling reflects the inefficiency of random exploration compared to ballistic motion (α = 1).

## 9. Summary

In [ ]:
print("=" * 60)
print("SUMMARY: Diffusive 1-D Simulation")
print("=" * 60)
print(f"""
RTM Prediction for Diffusive Regime: α = 2.00
Paper Reported Value: α ≈ 2.00

This Simulation:
  • Fitted exponent: α = {alpha:.4f} ± {std_err:.4f}
  • R² = {R_squared:.6f}
  • Deviation from theory: {abs(alpha - 2.0) / 2.0 * 100:.2f}%
  
INTERPRETATION:
The diffusive regime shows T ∝ L², meaning time scales quadratically
with distance. This is the classic random walk result where the
walker explores space inefficiently through Brownian motion.

In the RTM staircase of exponents:
  • Ballistic:  α ≈ 1   (straight-line motion)
  • Diffusive:  α ≈ 2   (random walk) ← THIS SIMULATION
  • Fractal:    α ≈ 2.5 (hierarchical structures)
  • Quantum:    α ≈ 3.5 (confined systems)

STATUS: {'✓ CONFIRMED' if abs(alpha - 2.0) < 0.1 else '⚠ NEEDS REVIEW'}
""")

## 10. Save Results

In [ ]:
# Save to CSV
df.to_csv('output/diffusive_1d_data.csv', index=False)
print("Data saved to output/diffusive_1d_data.csv")

# Display final dataframe
df